In [106]:
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import random
import os

# Function to generate a random color
#def random_color():
#    return tuple(random.randint(0, 255) for _ in range(4))

# Function to generate a vertical gradient

def random_color():
    return (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 255)

def generate_vertical_gradient(width, height):
    start_color = random_color()
    end_color = random_color()
    base = Image.new('RGBA', (width, height), start_color)
    top = Image.new('RGBA', (width, height), end_color)
    mask = Image.new('L', (width, height))
    mask_data = []
    for y in range(height):
        for x in range(width):
            mask_data.append(int(255 * (y / height)**2))  # Making the transition more abrupt
    mask.putdata(mask_data)
    vertical_gradient = Image.composite(base, top, mask)
    return vertical_gradient

def generate_radial_gradient(width, height):
    inner_color = random_color()
    outer_color = random_color()
    center_x, center_y = width // 2, height // 2
    max_radius = max(center_x, center_y)
    base = Image.new('RGBA', (width, height), outer_color)
    radial_gradient = Image.new('RGBA', (width, height))
    draw = ImageDraw.Draw(radial_gradient)
    for radius in range(max_radius, 0, -1):
        color = tuple([
            int(inner_color[i] + (outer_color[i] - inner_color[i]) * ((radius / max_radius)**3))
            for i in range(4)
        ])
        draw.ellipse((center_x - radius, center_y - radius, center_x + radius, center_y + radius), fill=color)
    combined_gradient = Image.alpha_composite(base, radial_gradient)
    return combined_gradient

# Function to apply glassmorphism effect
def apply_glassmorphism(image, blur_radius=10, opacity=128):
    blurred = image.filter(ImageFilter.GaussianBlur(blur_radius))
    overlay = Image.new('RGBA', image.size, (255, 255, 255, opacity))
    combined = Image.alpha_composite(blurred, overlay)
    return combined

# Function to place stickers
def place_sticker(background, sticker_path, position, sticker_width):
    sticker = Image.open(sticker_path).convert("RGBA") # Adjust size as needed
    sticker_height = int(sticker.height * (sticker_width / sticker.width))
    sticker = sticker.resize((sticker_width, sticker_height), Image.Resampling.LANCZOS)
    background.paste(sticker, position, sticker)
    return background

# Function to create fancy text
def create_fancy_text(draw, text, font_path, font_size, position):
    font = ImageFont.truetype(font_path, font_size)
    text_bbox = draw.textbbox(position, text, font=font)
    text_width, text_height = text_bbox[2] - text_bbox[0], text_bbox[3] - text_bbox[1]
    text_x, text_y = position
    shadow_offset = 5
    draw.text((text_x + shadow_offset, text_y + shadow_offset), text, font=font, fill=(0, 0, 0, 128))
    outline_range = 3
    for x in range(-outline_range, outline_range + 1):
        for y in range(-outline_range, outline_range + 1):
            draw.text((text_x + x, text_y + y), text, font=font, fill=(0, 0, 0, 255))
    draw.text((text_x, text_y), text, font=font, fill=(255, 0, 0, 252))

# Function to randomly select a sticker from a directory
def select_random_sticker(directory):
    stickers = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith(('png', 'jpg', 'jpeg'))]
    return random.choice(stickers) if stickers else None

# Function to create thumbnail
def create_thumbnail(video_title, creator_sticker_dir, milestone_sticker_path, output_path, font_path):
    width, height = 1280, 720

    # Generate random vertical and radial gradients
    vertical_gradient = generate_vertical_gradient(width, height)
    radial_gradient = generate_radial_gradient(width, height)

    # Combine gradients
    combined_gradient = Image.alpha_composite(vertical_gradient, radial_gradient)
    combined_gradient = apply_glassmorphism(combined_gradient)
    
    # Select random content creator sticker
    creator_sticker_path = select_random_sticker(creator_sticker_dir)
    if creator_sticker_path:
        sticker_width = int(combined_gradient.width * 0.6) 
        combined_gradient = place_sticker(combined_gradient, creator_sticker_path, (-30, 170), sticker_width)
    
    # Place milestone sticker
    sticker_width = int(combined_gradient.width * 0.43) 
    combined_gradient = place_sticker(combined_gradient, milestone_sticker_path, (700, height-600), sticker_width)

    # Add fancy text
    draw = ImageDraw.Draw(combined_gradient)
    create_fancy_text(draw, video_title, font_path, 60, (width // 2-600, height // 2 - 250 ))

    # Save the image
    combined_image = combined_gradient.convert("RGB")
    combined_image.save(output_path)

# Example usage
video_title = "बन्ना गीत : बन्नी तुम्हारी कुंवारी रह जाती"
creator_sticker_dir = 'creator'
milestone_sticker_path = 'views/100k.png'
output_path = 'thumbnail55.jpg'
font_path = 'Aparajita.ttf'  # Path to your font file
create_thumbnail(video_title, creator_sticker_dir, milestone_sticker_path, output_path, font_path)
